<a href="https://colab.research.google.com/github/IMOKURI/chaii-Hindi-and-Tamil-QA/blob/main/chaii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📔 About this notebook ...

[chaii - Hindi and Tamil Question Answering](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/)

# Memo



## ToDo

- [ ] [ラベルノイズ補正](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/264395)
    - [ ] [これもかな](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/266109)
- [x] [Post process でスコアアップ](https://www.kaggle.com/nbroad/chaii-qa-torch-5-fold-with-post-processing-765)
- モデル
    - [ ] [monsoon-nlp/hindi-tpu-electra](https://huggingface.co/monsoon-nlp/hindi-tpu-electra)
    - [ ] [RemBERT](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/267827)
    - [ ] [google/muril-base-cased](https://huggingface.co/google/muril-base-cased)
    - その他 [multilingual & QA models](https://huggingface.co/models?filter=multilingual&pipeline_tag=question-answering)
- [x] モデルクラスで  `AutoModelForQuestionAnswering` クラス を使ってみる

## Done


## Works Well


## Doesn't Work


## Not To Do

- [2つのモデルを作る](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/267604) - [経緯](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/264749)
- [位置によるペナルティを課す Loss](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/266832)


## Additional Datasets

Search from [here](https://www.kaggle.com/c/chaii-hindi-and-tamil-question-answering/discussion/264581).

- [External Data - MLQA, XQUAD Preprocessing](https://www.kaggle.com/rhtsingh/external-data-mlqa-xquad-preprocessing) hindi のみ
- [Squad_Translated_to_Tamil for Chaii](https://www.kaggle.com/msafi04/squad-translated-to-tamil-for-chaii) tamil のみ


## Reference Notebooks

- [ChAII - EDA & Baseline](https://www.kaggle.com/thedrcat/chaii-eda-baseline/)
- [chaii QA - 5 Fold XLMRoberta Torch | FIT](https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-fit/)
- [chaii QA - 5 Fold XLMRoberta Torch | Infer](https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-infer)



# Prepare for Colab

In [1]:
!nvidia-smi

Tue Sep 14 15:32:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import sys
import zipfile

if os.path.exists('init.txt'):
    print("Already initialized.")

else:
    if 'google.colab' in sys.modules:
        from google.colab import drive
        drive.mount('/content/drive')
        dataset_dir = "/content/drive/MyDrive/Datasets"

        # ====================================================
        # Competition datasets
        # ====================================================
        with zipfile.ZipFile(f"{dataset_dir}/chaii-hindi-and-tamil-question-answering.zip", "r") as zp:
            zp.extractall(path="./")
        #with zipfile.ZipFile(f"{dataset_dir}/chaii-external-data-mlqa-xquad-preprocessing.zip", "r") as zp:
        #    zp.extractall(path="./")
        #with zipfile.ZipFile(f"{dataset_dir}/chaii-Squad_Translated_to_Tamil.zip", "r") as zp:
        #    zp.extractall(path="./")

    # for StratifiedGroupKFold
    # !pip uninstall -y scikit-learn
    # !pip install --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn

    # for MultilabelStratifiedKFold
    # !pip install -q iterative-stratification

    # for CosineAnnealingWarmupRestarts
    # !pip install -qU 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'

    !pip install -q wandb
    # !pip install -q optuna

    # ====================================================
    # Competition specific libraries
    # ====================================================
    !pip install -q transformers
    !pip install -q sentencepiece
    # !pip install -q textstat
    # !pip install -q nlpaug

    !touch init.txt

Mounted at /content/drive
     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 97 kB 7.9 MB/s 
     |████████████████████████████████| 170 kB 83.5 MB/s 
     |████████████████████████████████| 133 kB 89.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 2.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 71.1 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 636 kB 83.7 MB/s 
     |████████████████████████████████| 3.3 MB 81.0 MB/s 
     |████████████████████████████████| 1.2 MB 4.1 MB/s 


# Library

In [3]:
# General libraries
import collections
import glob
import json
import math
import os
import random
import re
import statistics
import time
import warnings
from contextlib import contextmanager

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import torch
import torch.cuda.amp as amp
import torch.nn as nn
import torch.nn.functional as F
import wandb
# from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import mean_squared_error, jaccard_score
from sklearn.model_selection import KFold, StratifiedKFold  # , StratifiedGroupKFold
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

In [4]:
# Competition specific libraries
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nltk
# import textstat
import transformers as T

In [5]:
warnings.filterwarnings("ignore")

In [6]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')

In [7]:
netrc = "/content/drive/MyDrive/.netrc" if 'google.colab' in sys.modules else "../input/wandbtoken/.netrc"
!cp -f {netrc} ~/
!wandb login

wandb_tags = []

wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    wandb_tags.append(torch.cuda.get_device_name(0))

# Load Data

In [9]:
DATA_DIR = "./" if 'google.colab' in sys.modules else "../input/chaii-hindi-and-tamil-question-answering/"
OUTPUT_DIR = "./"
MODEL_DIR = "./models/"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

In [10]:
train = pd.read_csv(DATA_DIR + "train.csv")
test = pd.read_csv(DATA_DIR + "test.csv")
sub = pd.read_csv(DATA_DIR + "sample_submission.csv")

#external_squad_translated_tamil = pd.read_csv(DATA_DIR + "squad_translated_tamil.csv")
#external_mlqa = pd.read_csv(DATA_DIR + "mlqa_hindi.csv")
#external_xquad = pd.read_csv(DATA_DIR + "xquad.csv")


#  Config

In [11]:
# seed = random.randrange(10000)
seed = 440
print(seed)

440


In [12]:
class Config:
    wandb_entity = "imokuri"
    wandb_project = "chaii"
    print_freq = 100

    preprocess = False
    train = True
    validate = False
    inference = False

    debug = False
    num_debug_data = 50

    amp = False

In [13]:
config_defaults = {
    "seed": seed,
    # "n_class": 1,
    "n_fold": 5,
    "epochs": 2,
    "batch_size": 4,
    "gradient_accumulation_steps": 5,
    "max_grad_norm": 1.0,
    "criterion": "ChaiiCrossEntropyLoss",
    "optimizer": "BertAdamW",
    "scheduler": "get_cosine_schedule_with_warmup",
    "max_lr": 5e-5,
    "lr": 2e-5,
    "min_lr": 1e-5,
    "weight_decay": 0.01,
    "model_name": "deepset/xlm-roberta-large-squad2",
    # "model_name": "deepset/xlm-roberta-base-squad2",
    # "model_name": "google/rembert",
    "model_class": "bare", # bare, qa
    "max_len": 384,
    "doc_stride": 128,
    "dropout": 0.0,
    "init_weights": True,
    "init_layers": 1,
    # "freeze_layers": 0,
    "datasets": [
        "mlqa:v1",
        "xquad:v1",
        "squad_translated_tamil:v1",
    ],
    "models": [
        "base-models:v1",
    ]
}

In [14]:
if Config.debug:
    config_defaults["n_fold"] = 3
    config_defaults["epochs"] = 1
    Config.print_freq = 10

In [15]:
if Config.train:
    wandb_job_type = "training"

elif Config.inference:
    wandb_job_type = "inference"

elif Config.validate:
    wandb_job_type = "validation"

elif Config.preprocess:
    wandb_job_type = "preprocess"

else:
    wandb_job_type = ""

In [16]:
if Config.debug:
    wandb_tags.append("debug")
    
if Config.amp:
    wandb_tags.append("amp")

In [17]:
if Config.debug:
    run = wandb.init(
        entity=Config.wandb_entity,
        project=Config.wandb_project,
        config=config_defaults,
        tags=wandb_tags,
        mode="disabled",
    )
else:
    run = wandb.init(
        entity=Config.wandb_entity,
        project=Config.wandb_project,
        config=config_defaults,
        job_type=wandb_job_type,
        tags=wandb_tags,
        save_code=True,
    )

wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)


In [18]:
config = wandb.config

# Load Artifacts

In [19]:
if config.datasets != []:
    external_data = []
    for name_version in config.datasets:
        name, version = name_version.split(":")
        os.makedirs(name, exist_ok=True)

        if Config.debug:
            artifact_path = f"{Config.wandb_entity}/{Config.wandb_project}/{name_version}"
            api = wandb.Api()
            artifact = api.artifact(artifact_path)

        else:
            artifact_path = f"{name_version}"
            artifact = run.use_artifact(artifact_path)

        artifact.download(name)

        df = pd.read_csv(f"{name}/{name}.csv")
        external_data.append(df)

    external_train = pd.concat(external_data)

In [20]:
if Config.inference:
    api = wandb.Api()
    for artifact_id in config.models:
        name_version = artifact_id.replace(":", "-")
        if not os.path.exists(name_version):
            os.makedirs(name_version)

        for fold in range(config.n_fold):
            try:
                artifact_path = f"{Config.wandb_entity}/{Config.wandb_project}/{artifact_id}"
                artifact = api.artifact(artifact_path)
                artifact.download(name_version)
            except Exception as e:
                print(f"Error: {artifact_path}, {e}")

# EDA-1

In [21]:
for df in [train, test]:
    print(f"=" * 120)
    print(df.isnull().sum())

id              0
context         0
question        0
answer_text     0
answer_start    0
language        0
dtype: int64
id          0
context     0
question    0
language    0
dtype: int64


In [22]:
for df in [train, test]:
    print(f"=" * 120)
    print(df["language"].value_counts())

hindi    746
tamil    368
Name: language, dtype: int64
hindi    3
tamil    2
Name: language, dtype: int64


In [23]:
if config.datasets != []:
    print(external_train.isnull().sum())
    print(f"=" * 120)
    print(external_train["language"].value_counts())

context         0
question        0
answer_text     0
answer_start    0
language        0
dtype: int64
hindi    6615
tamil    3567
Name: language, dtype: int64


# Preprocessing

In [24]:
def convert_answers(row):
    return {'answer_start': [row[0]], 'text': [row[1]]}

In [25]:
def correct_labels(df):
    df.loc[df['id'] == '', 'answer_text'] = ''
    df.loc[df['id'] == '', 'answer_start'] = 0
    pass

In [26]:
def get_train_data(train):
    train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

    return train

In [27]:
def get_test_data(test):

    return test

In [28]:
train = get_train_data(train)

if config.datasets != []:
    external_train = get_train_data(external_train)

In [29]:
test = get_test_data(test)

### External Data

In [30]:
# 前処理
if False and Config.preprocess:
    external_squad_translated_tamil["language"] = "tamil"


In [31]:
# dataset 保存
if False and Config.preprocess:
    !mkdir -p squad_translated_tamil
    external_squad_translated_tamil.to_csv("squad_translated_tamil/squad_translated_tamil.csv", index=False)
    artifact = wandb.Artifact('squad_translated_tamil', type='dataset')
    artifact.add_dir("squad_translated_tamil/")
    run.log_artifact(artifact)

    !mkdir -p mlqa
    external_mlqa.to_csv("mlqa/mlqa.csv", index=False)
    artifact = wandb.Artifact('mlqa', type='dataset')
    artifact.add_dir("mlqa/")
    run.log_artifact(artifact)

    !mkdir -p xquad
    external_xquad.to_csv("xquad/xquad.csv", index=False)
    artifact = wandb.Artifact('xquad', type='dataset')
    artifact.add_dir("xquad/")
    run.log_artifact(artifact)


# EDA-2

In [32]:
for df in [train, test, sub]:
    print(f"=" * 120)
    df.info()
    display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114 entries, 0 to 1113
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1114 non-null   object
 1   context       1114 non-null   object
 2   question      1114 non-null   object
 3   answer_text   1114 non-null   object
 4   answer_start  1114 non-null   int64 
 5   language      1114 non-null   object
 6   answers       1114 non-null   object
dtypes: int64(1), object(6)
memory usage: 61.0+ KB


,id,context,question,answer_text,answer_start,language,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"{'answer_start': [53], 'text': ['206']}"
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,"{'answer_start': [2358], 'text': ['காசுமீரில்']}"
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil,"{'answer_start': [0], 'text': ['சர் அலெக்ஸாண்ட..."
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil,"{'answer_start': [68], 'text': ['தாலாட்டு']}"
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil,"{'answer_start': [585], 'text': ['சூரியனும்']}"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5 non-null      object
 1   context   5 non-null      object
 2   question  5 non-null      object
 3   language  5 non-null      object
dtypes: object(4)
memory usage: 288.0+ bytes


,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5 non-null      object 
 1   PredictionString  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes


,id,PredictionString
0,22bff3dec,NaN
1,282758170,NaN
2,d60987e0e,NaN
3,f99c770dc,NaN
4,40dec1964,NaN


# CV Split

In [33]:
if Config.debug:
    train = train.sample(n=Config.num_debug_data, random_state=config.seed).reset_index(drop=True)
    if config.datasets != []:
        external_train = external_train.sample(n=Config.num_debug_data, random_state=config.seed).reset_index(drop=True)
    if len(sub) > Config.num_debug_data:
        test = test.sample(n=Config.num_debug_data, random_state=config.seed).reset_index(drop=True)
        sub = sub.sample(n=Config.num_debug_data, random_state=config.seed).reset_index(drop=True)

In [34]:
Fold = StratifiedKFold(n_splits=config.n_fold, shuffle=True, random_state=seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train["language"])):
    train.loc[val_index, "fold"] = int(n)
train["fold"] = train["fold"].astype(np.int8)
print(train.groupby(["fold", "language"]).size())

fold  language
0     hindi       149
      tamil        74
1     hindi       149
      tamil        74
2     hindi       149
      tamil        74
3     hindi       150
      tamil        73
4     hindi       149
      tamil        73
dtype: int64


In [35]:
if config.datasets != []:
    external_train["fold"] = -1
    external_train['id'] = list(np.arange(1, len(external_train)+1))
    train = pd.concat([train, external_train]).reset_index(drop=True)

# Utils

In [36]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f"[{name}] start")
    yield
    LOGGER.info(f"[{name}] done in {time.time() - t0:.0f} s.")


def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True


seed_torch(seed=config.seed)

# Dataset

In [37]:
class BaseDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        super().__init__()

        self.tokenizer = T.AutoTokenizer.from_pretrained(model_name)

        self.features = []
        if include_labels:
            for i, row in df.iterrows():
                self.features += self.prepare_train_features(row)
        else:
            for i, row in df.iterrows():
                self.features += self.prepare_test_features(row)

        self.include_labels = include_labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, item):
        feature = self.features[item]

        if self.include_labels:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                # 'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

    def prepare_train_features(self, example):
        example["question"] = example["question"].lstrip()
        tokenized_example = self.tokenizer(
            example["question"],
            example["context"],
            truncation="only_second",
            max_length=config.max_len,
            stride=config.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
        offset_mapping = tokenized_example.pop("offset_mapping")

        features = []
        for i, offsets in enumerate(offset_mapping):
            feature = {}
            feature["example_id"] = example['id']
            feature['context'] = example['context']
            feature['question'] = example['question']

            input_ids = tokenized_example["input_ids"][i]
            attention_mask = tokenized_example["attention_mask"][i]

            feature['input_ids'] = input_ids
            feature['attention_mask'] = attention_mask
            feature['offset_mapping'] = offsets

            cls_index = input_ids.index(self.tokenizer.cls_token_id)
            sequence_ids = tokenized_example.sequence_ids(i)
            feature['sequence_ids'] = [0 if i is None else i for i in sequence_ids]

            sample_index = sample_mapping[i]
            answers = example["answers"]

            if len(answers["answer_start"]) == 0:
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1

                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    feature["start_position"] = cls_index
                    feature["end_position"] = cls_index
                else:
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    feature["start_position"] = token_start_index - 1
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    feature["end_position"] = token_end_index + 1

            features.append(feature)
        return features

    def prepare_test_features(self, example):
        example["question"] = example["question"].lstrip()
        tokenized_example = self.tokenizer(
            example["question"],
            example["context"],
            truncation="only_second",
            max_length=config.max_len,
            stride=config.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )

        features = []
        for i in range(len(tokenized_example["input_ids"])):
            feature = {}
            feature["example_id"] = example['id']
            feature['context'] = example['context']
            feature['question'] = example['question']
            feature['input_ids'] = tokenized_example['input_ids'][i]
            feature['attention_mask'] = tokenized_example['attention_mask'][i]
            feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
            feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
            features.append(feature)
        return features

In [38]:
if config.model_name != "":
    train_ds = BaseDataset(train, config.model_name)
    print(train_ds[0])
    print(len(train_ds))

Downloading:   0%|          | 0.00/179 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

{'input_ids': tensor([     0,  69535,  81049,  37368, 153264,  12095,  52989,  21883,   1629,
        145615,     32,      2,      2,   3219, 224013, 124335,   5966,  69535,
          4930,  74149,  12095,  52989,  21883, 182394,   3686,  51833,  57210,
        101912,     15,   6161,   2912,  70597,  52989,  21883, 102080,  54512,
         91585,   1962, 212933,  18599,  16242,  94236,     16, 198236,  29160,
         12095,  52989,  21883, 173139,  23618,  72817,      5,   5894, 198236,
         81049,  37334, 144257,   7827,  82890,  84853,  80517, 114452, 232094,
          3686,  17984,  11830,  62001, 182394,   4167,      5, 203312,  10753,
         50667,   2650,      4,  45303,   1962, 163062, 198236,  29160, 176030,
         15453,      4,   3219, 171093,   5944,   2650,   8120,  10175,  12095,
         52989,  21883,     15,   2650,  24183,   5638,  14861,     16,  56735,
          3219, 171093,   5944,   2650,  12009, 145578,  10832,   2802,   2650,
         26873,  52989,  2

In [39]:
if config.model_name != "":
    test_ds = BaseDataset(test, config.model_name, include_labels=False)
    print(test_ds[0])

{'input_ids': tensor([     0,      6,  38033,  91262,  20546,  85149,    471,  58380,    641,
          8062,   6004,    460,      2,      2,      6,  38033,  91262,  20546,
         85149,     15, 206327,     12,    361, 182198,  26819,     74,      6,
        196859,   1026,      4,  15297,     16,    967,   9261,  41162, 156793,
         64382,  46297, 103766,   1404,   7294,   1293,    125, 222600,   5725,
         11515,      6,  38033,  91262,  20546,  85149,    641,  22274,    361,
        182198,  26819,    629,      6, 196859,   1026,      4,  15297,    421,
         11645,   3813,    125,  24939,  62573,  39477,      5, 233097,   4429,
         45951,  25594,    871,  56980,   4149,   1471,    998,  23263,    646,
          1293,    125,  35056,  56980,   4149,   1471,    998,  20546,  85149,
         47211,  12797,  27193,    421,   5564, 220307,   9331,    287,   3765,
          3946,  13430, 120116,    125,      6,  38033,  91262,  20546,  85149,
           471, 222600,   

# 🚗 Model

In [40]:
if config.model_name != "":
    print(T.AutoConfig.from_pretrained(config.model_name))

XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



In [41]:
class BaseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.auto_config = T.AutoConfig.from_pretrained(model_name)
        self.auto_config.update({
            "hidden_dropout_prob": config.dropout,
            "layer_norm_eps": 1e-7,
        })

        self.auto_model = T.AutoModel.from_pretrained(model_name, config=self.auto_config, add_pooling_layer=False)
        self.qa_outputs = nn.Linear(self.auto_config.hidden_size, 2)

        if config.init_weights:
            self._init_weights(self.qa_outputs)

        if config.init_layers > 0:
            self._init_layers()
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.auto_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def _init_layers(self):
        # re-init pooler
        # self.auto_model.pooler.dense.weight.data.normal_(mean=0.0, std=self.auto_model.config.initializer_range)
        # self.auto_model.pooler.dense.bias.data.zero_()
        # for p in self.auto_model.pooler.parameters():
        #     p.requires_grad = True

        # re-init encoder
        layers = self.auto_model.encoder.layer[-config.init_layers:]
        for layer in layers:
            for module in layer.modules():
                if isinstance(module, nn.Linear):
                    # Slightly different from the TF version which uses truncated_normal for initialization
                    # cf https://github.com/pytorch/pytorch/pull/5617
                    module.weight.data.normal_(mean=0.0, std=self.auto_model.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                    module.weight.data.normal_(mean=0.0, std=self.auto_model.config.initializer_range)
                    if module.padding_idx is not None:
                        module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
    ):
        outputs = self.auto_model(
            input_ids,
            attention_mask,
        )

        last_hidden_state = outputs[0]
        
        qa_logits = self.qa_outputs(last_hidden_state)
        
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
        return start_logits, end_logits

In [42]:
class QAModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.auto_config = T.AutoConfig.from_pretrained(model_name)
        self.auto_config.update({
            "hidden_dropout_prob": config.dropout,
            # "layer_norm_eps": 1e-7,
        })

        self.auto_model = T.AutoModelForQuestionAnswering.from_pretrained(model_name, config=self.auto_config)

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
    ):
        outputs = self.auto_model(
            input_ids,
            attention_mask,
        )

        return outputs.start_logits, outputs.end_logits

In [43]:
if config.model_name != "":
    if config.model_class == "bare":
        model = BaseModel(config.model_name)
    elif config.model_class == "qa":
        model = QAModel(config.model_name)
    print(model)

    train_dataset = BaseDataset(train, config.model_name)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, drop_last=True)

    for features in train_loader:
        output = model(features["input_ids"], features["attention_mask"])
        print(output)
        break

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaModel: ['qa_outputs.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'qa_outputs.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModel(
  (auto_model): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm)

In [44]:
if config.model_name != "":
    for n, (name, tensor) in enumerate(list(model.named_parameters())):
        print(f"{n:>4}: {tensor.requires_grad}, {name}")

   0: True, auto_model.embeddings.word_embeddings.weight
   1: True, auto_model.embeddings.position_embeddings.weight
   2: True, auto_model.embeddings.token_type_embeddings.weight
   3: True, auto_model.embeddings.LayerNorm.weight
   4: True, auto_model.embeddings.LayerNorm.bias
   5: True, auto_model.encoder.layer.0.attention.self.query.weight
   6: True, auto_model.encoder.layer.0.attention.self.query.bias
   7: True, auto_model.encoder.layer.0.attention.self.key.weight
   8: True, auto_model.encoder.layer.0.attention.self.key.bias
   9: True, auto_model.encoder.layer.0.attention.self.value.weight
  10: True, auto_model.encoder.layer.0.attention.self.value.bias
  11: True, auto_model.encoder.layer.0.attention.output.dense.weight
  12: True, auto_model.encoder.layer.0.attention.output.dense.bias
  13: True, auto_model.encoder.layer.0.attention.output.LayerNorm.weight
  14: True, auto_model.encoder.layer.0.attention.output.LayerNorm.bias
  15: True, auto_model.encoder.layer.0.intermed

# Optimizer

In [45]:
def bert_optimizer(model):
    named_parameters = list(model.named_parameters())    

    if (
        "base" in config.model_name
        or "L-12" in config.model_name
    ):
        bert_parameters = named_parameters[:197]    
        regressor_parameters = named_parameters[197:]
        second_block = 69
        third_block = 133

    elif (
        "large" in config.model_name
        or "L-24" in config.model_name
    ):
        bert_parameters = named_parameters[:389]    
        regressor_parameters = named_parameters[389:]
        second_block = 133
        third_block = 261

    elif "rembert" in config.model_name:
        bert_parameters = named_parameters[:519]
        regressor_parameters = named_parameters[519:]
        second_block = 199
        third_block = 359
        
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(bert_parameters):
        weight_decay = 0.0 if "bias" in name else config.weight_decay

        if layer_num >= third_block:
            lr = config.max_lr
        elif layer_num >= second_block:
            lr = config.lr
        else:
            lr = config.min_lr

        parameters.append({"params": params, "weight_decay": weight_decay, "lr": lr})

    return T.AdamW(parameters, eps=1e-7)

# Loss

In [46]:
def chaii_cross_entropy(preds, labels):
    start_preds, end_preds = preds
    start_labels, end_labels = labels
    
    start_loss = nn.CrossEntropyLoss(ignore_index=-1)(start_preds, start_labels)
    end_loss = nn.CrossEntropyLoss(ignore_index=-1)(end_preds, end_labels)
    total_loss = (start_loss + end_loss) / 2
    return total_loss

# Scoring

In [47]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [48]:
def get_result(result_df, fold=config.n_fold):
    score = result_df["jaccard"].mean()
    LOGGER.info(f"Score: {score:<.5f}")
    if fold == config.n_fold:
        wandb.log({"CV": score})
    else:
        wandb.log({f"CV_fold{fold}": score})

# Helper Functions

In [49]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

In [50]:
def compute_grad_norm(parameters, norm_type=2.0):
    """Refer to torch.nn.utils.clip_grad_norm_"""
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    parameters = [p for p in parameters if p.grad is not None]
    norm_type = float(norm_type)
    total_norm = 0
    for p in parameters:
        param_norm = p.grad.data.norm(norm_type)
        total_norm += param_norm.item() ** norm_type
    total_norm = total_norm ** (1. / norm_type)
    return total_norm

In [51]:
def train_fn(train_loader, model, criterion, optimizer, scheduler, scaler, fold, epoch, device):
    losses = AverageMeter()

    # switch to train mode
    model.train()
    start = time.time()
    optimizer.zero_grad()

    for step, features in enumerate(train_loader):
        input_ids = features["input_ids"].to(device)
        attention_mask = features["attention_mask"].to(device)
        labels_start = features["start_position"].to(device)
        labels_end = features["end_position"].to(device)
        batch_size = labels_start.size(0)

        with amp.autocast(enabled=Config.amp):
            out_start, out_end = model(input_ids, attention_mask)
            loss = criterion((out_start, out_end), (labels_start, labels_end))
            losses.update(loss.item(), batch_size)
            loss = loss / config.gradient_accumulation_steps
            
        scaler.scale(loss).backward()

        if (step + 1) % config.gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()
            optimizer.zero_grad()
        else:
            grad_norm = compute_grad_norm(model.parameters())

        end = time.time()
        if step % Config.print_freq == 0 or step == (len(train_loader) - 1):
            print(
                f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                f"Loss: {losses.avg:.4f} "
                f"Grad: {grad_norm:.4f} "
                f"LR: {scheduler.get_lr()[0]:.6f} "
            )
            # wandb.log({
            #     "step": (epoch) * len(train_loader) + step,
            #     f"loss/fold{fold}": losses.avg,
            #     f"grad/fold{fold}": grad_norm,
            #     f"lr/fold{fold}": scheduler.get_lr()[0],
            # })

    return losses.avg

In [52]:
def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds_start = []
    preds_end = []
    start = time.time()

    for step, features in enumerate(valid_loader):
        input_ids = features["input_ids"].to(device)
        attention_mask = features["attention_mask"].to(device)
        labels_start = features["start_position"].to(device)
        labels_end = features["end_position"].to(device)
        batch_size = labels_start.size(0)

        # compute loss
        with torch.no_grad():
            out_start, out_end = model(input_ids, attention_mask)

        loss = criterion((out_start, out_end), (labels_start, labels_end))
        losses.update(loss.item(), batch_size)

        preds_start.append(out_start.to("cpu").numpy())
        preds_end.append(out_end.to("cpu").numpy())
        # preds.append(y_preds.softmax(1).to("cpu").numpy())
        # preds.append(y_preds.to("cpu").numpy())

        end = time.time()
        if step % Config.print_freq == 0 or step == (len(valid_loader) - 1):
            print(
                f"EVAL: [{step}/{len(valid_loader)}] "
                f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                f"Loss: {losses.avg:.4f} "
            )

    predictions_start = np.concatenate(preds_start)
    predictions_end = np.concatenate(preds_end)
    return losses.avg, predictions_start, predictions_end

# Postprocessing

In [53]:
def postprocess_qa_predictions(examples, features, tokenizer, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [54]:
# https://www.kaggle.com/nbroad/chaii-qa-torch-5-fold-with-post-processing-765
def postpurocess_by_nbroad(preds_df):
    bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
    bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

    cleaned_preds = []
    for pred, context in preds_df[["prediction", "context"]].to_numpy():
        if pred == "":
            cleaned_preds.append(pred)
            continue
        while any([pred.startswith(y) for y in bad_starts]):
            pred = pred[1:]
        while any([pred.endswith(y) for y in bad_endings]):
            if pred.endswith("..."):
                pred = pred[:-3]
            else:
                pred = pred[:-1]

        cleaned_preds.append(pred)

    preds_df["prediction"] = cleaned_preds

    return preds_df

# Train Loop

In [55]:
def train_loop(df, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    trn_idx = df[df["fold"] != fold].index
    val_idx = df[df["fold"] == fold].index

    train_folds = df.loc[trn_idx].reset_index(drop=True)
    valid_folds = df.loc[val_idx].reset_index(drop=True)

    train_dataset = BaseDataset(train_folds, config.model_name)
    valid_dataset = BaseDataset(valid_folds, config.model_name)

    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        drop_last=False,
    )

    # ====================================================
    # Optimizer
    # ====================================================
    def get_optimizer(model):
        if config.optimizer == "Adam":
            optimizer = Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        elif config.optimizer == "AdamW":
            optimizer = T.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        elif config.optimizer == "BertAdamW":
            optimizer = bert_optimizer(model)
        return optimizer

    # ====================================================
    # Scheduler
    # ====================================================
    def get_scheduler(optimizer):
        # num_data = len(train_folds)
        num_data = len(train_dataset)
        num_steps = num_data // (config.batch_size * config.gradient_accumulation_steps) * config.epochs

        if config.scheduler == "CosineAnnealingWarmRestarts":
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=num_steps, T_mult=1, eta_min=config.min_lr, last_epoch=-1)
        elif config.scheduler == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(optimizer, T_max=num_steps, eta_min=config.min_lr, last_epoch=-1)
        elif config.scheduler == "CosineAnnealingWarmupRestarts":
            scheduler = CosineAnnealingWarmupRestarts(
                optimizer, first_cycle_steps=num_steps, max_lr=config.lr, min_lr=config.min_lr, warmup_steps=(num_steps // 10)
            )
        elif config.scheduler == "get_cosine_schedule_with_warmup":
            scheduler = T.get_cosine_schedule_with_warmup(
                optimizer, num_training_steps=num_steps, num_warmup_steps=(num_steps // 10)
            )
        return scheduler

    # ====================================================
    # Model
    # ====================================================
    if config.model_class == "bare":
        model = BaseModel(config.model_name)
    elif config.model_class == "qa":
        model = QAModel(config.model_name)
    model.to(device)

    optimizer = get_optimizer(model)
    scaler = amp.GradScaler(enabled=Config.amp)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # Criterion
    # ====================================================
    def get_criterion():
        if config.criterion == "CrossEntropyLoss":
            criterion = nn.CrossEntropyLoss()
        elif config.criterion == "BCEWithLogitsLoss":
            criterion = nn.BCEWithLogitsLoss()
        elif config.criterion == "MSELoss":
            criterion = nn.MSELoss()
        elif config.criterion == "ChaiiCrossEntropyLoss":
            criterion = chaii_cross_entropy
        return criterion

    criterion = get_criterion()

    # ====================================================
    # Loop
    # ====================================================
    best_score = -1
    best_loss = np.inf
    best_preds = None

    wandb.watch(model, log_freq=Config.print_freq)

    for epoch in range(config.epochs):
        start_time = time.time()

        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, scheduler, scaler, fold, epoch, device)

        # eval
        avg_val_loss, preds_start, preds_end = valid_fn(valid_loader, model, criterion, device)

        # postprocess 1
        predictions = postprocess_qa_predictions(
            valid_folds, valid_dataset.features, valid_dataset.tokenizer, (preds_start, preds_end)
        )

        oof_df = valid_folds[["id", "context", "answer_text"]]
        oof_df["prediction"] = oof_df['id'].apply(lambda r: predictions[r])

        # postprocess 2
        oof_df = postpurocess_by_nbroad(oof_df)

        # scoring
        oof_df['jaccard'] = oof_df[['answer_text', 'prediction']].apply(jaccard, axis=1)
        score = oof_df["jaccard"].mean()

        elapsed = time.time() - start_time
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Time: {elapsed:.0f}s")
        wandb.log({
            "epoch": epoch + 1,
            f"val_loss/fold{fold}": avg_val_loss,
            f"score/fold{fold}": score,
        })
        if avg_val_loss < best_loss:
            best_score = score
            best_loss = avg_val_loss
            best_preds = predictions
            LOGGER.info(f"Epoch {epoch+1} - Save Best Model. score: {best_score:.4f}, loss: {best_loss:.4f}")

            model_subdir = MODEL_DIR + f"fold{fold}/"
            os.makedirs(model_subdir, exist_ok=True)
            torch.save(model.state_dict(), f"{model_subdir}/pytorch_model.bin")
            with open(f'{model_subdir}/preds.json', 'w') as f:
                f.write(json.dumps(predictions, sort_keys=True, indent=4, ensure_ascii=False))
            model.auto_config.save_pretrained(model_subdir)
            train_dataset.tokenizer.save_pretrained(model_subdir)

    valid_folds["prediction"] = valid_folds['id'].apply(lambda r: best_preds[r])
    valid_folds['jaccard'] = valid_folds[['answer_text', 'prediction']].apply(jaccard, axis=1)

    return valid_folds, best_score, best_loss

# 🚀 Main

In [56]:
def main():
    # ====================================================
    # Training
    # ====================================================
    if Config.train:
        oof_df = pd.DataFrame()
        oof_result = []
        for fold in range(config.n_fold):
            seed_torch(seed + fold)

            _oof_df, score, loss = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            oof_result.append([fold, score, loss])

            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df, fold)

        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        
        loss = statistics.mean([d[2] for d in oof_result])
        wandb.log({"loss": loss})

        table = wandb.Table(data=oof_result, columns = ["fold", "score", "loss"])
        run.log({"Fold Result": table})
        
        # save result
        oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)
        wandb.save(OUTPUT_DIR + "oof_df.csv")

        artifact = wandb.Artifact(config.model_name.replace('/', '-'), type='model')
        artifact.add_dir(MODEL_DIR)
        run.log_artifact(artifact)


In [ ]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========
Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaModel: ['qa_outputs.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'qa_outputs.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: [1][0/5982] Elapsed 0m 1s (remain 116m 50s) Loss: 6.1654 Grad: 5.6987 LR: 0.000000 
Epoch: [1][100/5982] Elapsed 0m 58s (remain 56m 40s) Loss: 5.5192 Grad: 7.9162 LR: 0.000084 
Epoch: [1][200/5982] Elapsed 1m 54s (remain 54m 59s) Loss: 3.8947 Grad: 6.1532 LR: 0.000167 
Epoch: [1][300/5982] Elapsed 2m 51s (remain 53m 48s) Loss: 2.9398 Grad: 16.4977 LR: 0.000251 
Epoch: [1][400/5982] Elapsed 3m 47s (remain 52m 45s) Loss: 2.4894 Grad: 14.4766 LR: 0.000335 
Epoch: [1][500/5982] Elapsed 4m 43s (remain 51m 43s) Loss: 2.1770 Grad: 13.4975 LR: 0.000418 
Epoch: [1][600/5982] Elapsed 5m 40s (remain 50m 44s) Loss: 1.9631 Grad: 15.2799 LR: 0.000502 
Epoch: [1][700/5982] Elapsed 6m 36s (remain 49m 46s) Loss: 1.8082 Grad: 10.4813 LR: 0.000586 
Epoch: [1][800/5982] Elapsed 7m 32s (remain 48m 48s) Loss: 1.6902 Grad: 19.0783 LR: 0.000669 
Epoch: [1][900/5982] Elapsed 8m 29s (remain 47m 50s) Loss: 1.6073 Grad: 0.6474 LR: 0.000753 
Epoch: [1][1000/5982] Elapsed 9m 25s (remain 46m 53s) Loss: 1.5379

In [ ]:
wandb.finish()

In [ ]:
!nvidia-smi